In [ ]:
import json
import requests
import pandas as pd
import datetime
from sqlalchemy import create_engine
from pandas import DataFrame as df
import numpy as np
from oauth2client.service_account import ServiceAccountCredentials
import gspread
from clickhouse_key import ch_key
from clickhouse_driver import Client

In [ ]:
pd.set_option('display.max_columns', None)

# ONLINE DEAL

In [ ]:
client = Client(ch_key['db_host'],
               user=ch_key['db_user'],
               password=ch_key['db_password'],
               verify=False,
               database=ch_key['db_name'], settings={'use_numpy': True})

In [ ]:
def df(result, columns):
    col = []
    for i in columns:
        col.append(i[0])
        df = pd.DataFrame(result,columns=col)
    return df    
        
def date(date):
    date  = (datetime.datetime.strptime(date['date'][0],"%Y-%m-%d").date()+datetime.timedelta(days=1)) 
    return date

In [ ]:
result, columns = client.execute('''SELECT CAST(MAX(DATE(date)) as String) as date 
FROM messnerav.ONLINE_DEAL_DASH_BANNERS ''', with_column_types = True)
data = df(result, columns)
start = date(data)
start

In [ ]:
re, columns = client.execute(f'''

WITH A AS (
SELECT
date
, COUNT(DISTINCT CASE WHEN match(pagepath, '.*nedvizhimost/prod.*kvart.*') THEN dimension4 END) AS cards
, COUNT(DISTINCT CASE WHEN match(pagepath, '.*nedvizhimost/prod.*kvart.*') THEN dimension1 END) AS cards_us
, COUNT(DISTINCT CASE WHEN match(pagepath, '.*offer-placement/create/.*') THEN dimension4 END) AS placement
, COUNT(DISTINCT CASE WHEN match(pagepath, '.*offer-placement/create/.*') THEN dimension1 END) AS placement_us
FROM ga.PAGE_VIEWS  left join (
select dimension1, dimension4
from ga.UA_TRAFIC_BIG WHERE date >= '{start}') CR USING(dimension4)
WHERE
(match(pagepath, '.*nedvizhimost/prod.*kvart.*') OR match(pagepath, '.*offer-placement/create/.*'))
AND date >= '{start}'
GROUP BY date),

B AS (
SELECT
  date,
  coalesce(SUM(CASE WHEN (match(eventlabel, 'UPBANNER.*advert.*') AND eventAction LIKE 'online_deal%')  THEN uniqueEvents END),0) as lk_banners
, coalesce(SUM(CASE WHEN (match(eventlabel, 'UPBANNER.*sell.*more') AND eventAction LIKE 'online_deal|icons') THEN uniqueEvents END), 0) as card_banner
FROM ga.RAW_EVENTS
WHERE (match(eventlabel, 'UPBANNER.*advert.*') AND eventAction LIKE 'online_deal%')
OR
(match(eventlabel, 'UPBANNER.*sell.*more') AND eventAction LIKE 'online_deal|icons') AND  date >= '{start}'
GROUP BY date order by date desc)

select A.date as date, cards, cards_us, placement,placement_us, lk_banners, card_banner
from A left join
B on A.date=B.date
WHERE A.date >= '{start}' ORDER BY date DESC''', with_column_types = True)

In [ ]:
col = []
for i in columns:
    col.append(i[0])
res = pd.DataFrame(re,columns=col)
res.head(5)

In [ ]:
res['date'] = pd.to_datetime(res['date'], format='%Y-%m-%d')
res.info()

In [ ]:
# client.execute(
# "CREATE TABLE messnerav.ONLINE_DEAL_DASH_BANNERS(date Date, cards Nullable(Int), cards_us Nullable(Int), placement Nullable(Int), placement_us Nullable(Int), lk_banners Nullable(Int), card_banner Nullable(Int)) ENGINE = MergeTree() ORDER BY (date)"
# )

In [ ]:
client.insert_dataframe("INSERT INTO messnerav.ONLINE_DEAL_DASH_BANNERS VALUES", res) # Всегда должно быть <=7

# QUIZE

In [ ]:
client.execute(f''' ALTER TABLE messnerav.QUIZ_FULL_REPORT DELETE WHERE date = (select max(date) from messnerav.QUIZ_FULL_REPORT) ''')

In [ ]:
re, columns = client.execute(f'''

WITH
    (SELECT max(date) + 1
     FROM messnerav.QUIZ_FULL_REPORT) AS DATE1,
    yesterday() AS DATE2,
    SESSIONS AS
        (SELECT CAST(dateHourMinute, 'DATE')                                              AS date,
                dimension1,
                dimension4,
                lower(multiIf(extract(landingpagepath, 'quiz_source=("[^"]+"|\\w+)') = '', 'Inhouse',
                              extract(landingpagepath, 'quiz_source=("[^"]+"|\\w+)')))    AS partner,
                multiIf(landingpagepath LIKE '%novostroyki/quiz%', 'Квиз', landingpagepath LIKE '%novostroyki%',
                        'Новостройки', 'Другие страницы')                                 AS NB_LP,
                multiIf((source LIKE '%vk%') AND (medium = 'referral'), 'vk.com', source) AS source,
                medium,
                campaign,
                deviceCategory
         FROM ga.UA_TRAFIC_BIG
         WHERE (date >= DATE1)
           AND (date <= DATE2)
         GROUP BY date,
                  dimension1,
                  dimension4,
                  partner,
                  NB_LP,
                  source,
                  medium,
                  campaign,
                  deviceCategory),
    EVENTS AS
        (SELECT dimension4,
                eventCategory                              AS quiz_id,
                sum(multiIf(eventlabel = '1', 1, 0))       AS quize_started,
                sum(multiIf(eventlabel = 'success', 1, 0)) AS quize_succes
         FROM ga.RAW_EVENTS
         WHERE (eventAction = 'Step')
           AND ((date >= DATE1) AND (date <= DATE2))
         GROUP BY dimension4,
                  quiz_id),
    SALES AS
        (SELECT Date          AS CALL_DATE,
                caller,
                SOLDNESS      AS sale_state,
                max(SOLD_SUM) AS sold_sum,
                1             AS first
         FROM mart.NB_CALLS
         WHERE (SOLDNESS = 'Продан')
           AND ((Date >= DATE1) AND (Date <= DATE2))
         GROUP BY CALL_DATE,
                  caller,
                  sale_state),
    MAIL_LEADS AS
        (SELECT CAST(date, 'DATE')                                       AS LEAD_DATE,
                ID,
                PHONE,
                utm_source,
                utm_medium,
                utm_campaign,
                row_number() OVER (PARTITION BY PHONE ORDER BY date ASC) AS first
         FROM external.MAIL_DATA
         WHERE (CAST(date, 'DATE') >= DATE1)
           AND (CAST(date, 'DATE') <= DATE2)
         GROUP BY date,
                  ID,
                  PHONE,
                  utm_source,
                  utm_medium,
                  utm_campaign),
    LEEDS AS
        (SELECT LEAD_DATE,
                ID,
                utm_source,
                utm_medium,
                utm_campaign
         FROM MAIL_LEADS
                  LEFT JOIN SALES ON (PHONE = caller) AND (MAIL_LEADS.first = SALES.first)
         WHERE MAIL_LEADS.LEAD_DATE <= SALES.CALL_DATE
         GROUP BY LEAD_DATE,
                  ID,
                  utm_source,
                  utm_medium,
                  utm_campaign)
SELECT ifNull(SESSIONS.date, LEEDS.LEAD_DATE)                          AS date,
       coalesce(SESSIONS.source, LEEDS.utm_source, 'Неопределено')     AS utm_source,
       coalesce(SESSIONS.medium, LEEDS.utm_medium, 'Неопределено')     AS utm_medium,
       coalesce(SESSIONS.campaign, LEEDS.utm_campaign, 'Неопределено') AS utm_campaign,
       coalesce(SESSIONS.deviceCategory, 'Неопределено')               AS deviceCategory,
       COUNTDistinct(dimension1)                                       AS user_ids,
       sum(quize_started)                                              AS quize_started_c,
       sum(quize_succes)                                               AS quize_finished
FROM SESSIONS
         LEFT JOIN EVENTS ON SESSIONS.dimension4 = EVENTS.dimension4
         LEFT JOIN LEEDS ON quiz_id = ID
GROUP BY date,
         utm_source,
         utm_medium,
         utm_campaign,
         deviceCategory
ORDER BY date ASC ''', with_column_types = True)

In [ ]:
col = []
for i in columns:
    col.append(i[0])
res = pd.DataFrame(re,columns=col)
res.tail(5)

In [ ]:
client.insert_dataframe("INSERT INTO messnerav.QUIZ_FULL_REPORT VALUES", res)

# AUTH\REG TECH REP

In [ ]:
client.execute(f''' ALTER TABLE messnerav.REG_AUTH_TECH_REPORT DELETE WHERE date = (select max(date) from messnerav.REG_AUTH_TECH_REPORT) ''')

In [ ]:
re, columns = client.execute(f'''

SELECT * EXCEPT (Q1, Q2)
FROM (
         SELECT row_number() OVER (ORDER BY date ASC, b.eventAction ASC) AS Q1,
                *
         FROM (
                  WITH a AS
                           (SELECT CAST(dateHourMinute, 'DATE') AS date,
                                   eventAction,
                                   ifNull(sum(uniqueEvents), 0) AS u1,
                                   dimension4
                            FROM ga.RAW_EVENTS
                            WHERE match(eventlabel, '(NewAuthPhoneChecked)')
                              AND ((date >= (SELECT max(date) + 1
                                             FROM messnerav.REG_AUTH_TECH_REPORT)) AND (date <= yesterday()))
                              AND match(eventAction, '(FL|P_AGENT|PARTNER)')
                              AND match(pagepath,
                                        '(/login/$|/login$|offer-placement|/login/client$|/login/professional$|/login/partner$)')
                            GROUP BY date,
                                     eventAction,
                                     dimension4
                            ORDER BY date ASC,
                                     eventAction ASC),
                       b AS
                           (SELECT CAST(dateHourMinute, 'DATE') AS date,
                                   eventAction,
                                   ifNull(sum(uniqueEvents), 0) AS u2,
                                   dimension4
                            FROM ga.RAW_EVENTS
                            WHERE match(eventlabel, '(NewAuthSuccessful)')
                              AND ((date >= (SELECT max(date) + 1
                                             FROM messnerav.REG_AUTH_TECH_REPORT)) AND (date <= yesterday()))
                              AND match(eventAction, '(FL|P_AGENT|PARTNER)')
                              AND match(pagepath,
                                        '(/login/$|/login$|offer-placement|/login/client$|/login/professional$|/login/partner$|/reset-password)')
                            GROUP BY date,
                                     eventAction,
                                     dimension4
                            ORDER BY date ASC,
                                     eventAction ASC)
                  SELECT b.date                                                                                      AS date,
                         b.eventAction,
                         sum(multiIf(match(b.eventAction, '(FL|P_AGENT)'), u1, 0))                                   AS auth_phone_check,
                         COUNTDistinct(multiIf(match(b.eventAction, '(FL|P_AGENT)') AND (a.dimension4 = b.dimension4),
                                               a.dimension4, match(b.eventAction, '(PARTNER)'), b.dimension4,
                                               NULL))                                                                AS auth_success
                  FROM a
                           RIGHT JOIN b ON a.dimension4 = b.dimension4
                  WHERE b.eventAction IS NOT NULL
                  GROUP BY date,
                           b.eventAction
                  ORDER BY date ASC,
                           b.eventAction ASC
                  )
         ) AS t1
         FULL OUTER JOIN
     (
         SELECT row_number() OVER (ORDER BY date ASC, eventAction ASC) AS Q2,
                * EXCEPT (date, eventAction)
         FROM (
                  SELECT CAST(dateHourMinute, 'DATE')  AS date,
                         eventAction,
                         sum(multiIf((eventlabel = 'NewReg_AuthRoleChoose') AND
                                     match(eventAction, '(FL|P_AGENT|PARTNER)') AND match(pagepath,
                                                                                          '(/login/$|/login$|offer-placement|/login/client$|/login/professional$|/login/partner$)'),
                                     uniqueEvents, 0)) AS role_choose,
                         sum(multiIf((eventlabel = 'NewRegPhoneChecked') AND match(eventAction, '(FL|P_AGENT)') AND
                                     match(pagepath,
                                           '(/login/$|/login$|offer-placement|/login/client$|/login/professional$|/login/partner$)'),
                                     uniqueEvents, 0)) AS phone_checked,
                         sum(multiIf((eventlabel = 'NewRegSmsInput') AND match(eventAction, '(FL|P_AGENT)') AND
                                     match(pagepath,
                                           '(/login/$|/login$|offer-placement|/login/client$|/login/professional$|/login/partner$)'),
                                     uniqueEvents, 0)) AS sms_input,
                         sum(multiIf((eventlabel = 'NewRegSuccess') AND match(eventAction, '(FL|P_AGENT)') AND
                                     match(pagepath,
                                           '(/login/$|/login$|offer-placement|/login/client$|/login/professional$|/login/partner$)'),
                                     uniqueEvents, 0)) AS reg_success,
                         sum(multiIf((eventlabel = 'NewRegPassSetup') AND match(eventAction, '(FL|P_AGENT)') AND
                                     match(pagepath,
                                           '(/login/$|/login$|offer-placement|/login/client$|/login/professional$|/login/partner$)'),
                                     uniqueEvents, 0)) AS pass_setup,
                         sum(multiIf((eventlabel = 'NewRegEmailInput') AND match(eventAction, '(FL|P_AGENT)') AND
                                     match(pagepath,
                                           '(/login/$|/login$|offer-placement|/login/client$|/login/professional$|/login/partner$)'),
                                     uniqueEvents, 0)) AS email_input,
                         sum(multiIf((eventlabel = 'NewRegSuccessful') AND match(eventAction, '(FL|P_AGENT)') AND
                                     match(pagepath,
                                           '(/login/$|/login$|offer-placement|/login/client$|/login/professional$|/login/partner$)'),
                                     uniqueEvents, 0)) AS reg_success_full,
                         sum(multiIf((eventlabel = 'NewRegRequestStart') AND match(eventAction, '(PARTNER)') AND
                                     match(pagepath,
                                           '(/login/$|/login$|offer-placement|/login/client$|/login/professional$|/login/partner$)'),
                                     uniqueEvents, 0)) AS reg_partner_req_start,
                         sum(multiIf((eventlabel = 'NewRegRequestSend') AND match(eventAction, '(PARTNER)') AND
                                     match(pagepath,
                                           '(/login/$|/login$|offer-placement|/login/client$|/login/professional$|/login/partner$)'),
                                     uniqueEvents, 0)) AS reg_partner_req_send,
                         sum(multiIf((eventlabel = 'NewReg_AuthForgotPass') AND
                                     match(eventAction, '(FL|P_AGENT|PARTNER)') AND match(pagepath,
                                                                                          '(/login/$|/login$|offer-placement|/login/client$|/login/professional$|/login/partner$)'),
                                     uniqueEvents, 0)) AS REG_AUTH_er_ALL_roles_forgot_pass_no_phone_email_first,
                         sum(multiIf((eventlabel = 'NewRegEmailWrongCode') AND match(eventAction, '(FL|P_AGENT)') AND
                                     match(pagepath,
                                           '(/login/$|/login$|offer-placement|/login/client$|/login/professional$|/login/partner$|/reset-password)'),
                                     uniqueEvents, 0)) AS er_fl_chmak_email_wrong_code,
                         sum(multiIf((eventlabel = 'NewRegPhoneChange') AND match(eventAction, '(FL|P_AGENT)') AND
                                     match(pagepath,
                                           '(/login/$|/login$|offer-placement|/login/client$|/login/professional$|/login/partner$|/reset-password)'),
                                     uniqueEvents, 0)) AS er_fl_chmak_cnahge_phone,
                         sum(multiIf((eventlabel = 'NewRegEmailChange') AND match(eventAction, '(FL|P_AGENT)') AND
                                     match(pagepath,
                                           '(/login/$|/login$|offer-placement|/login/client$|/login/professional$|/login/partner$|/reset-password)'),
                                     uniqueEvents, 0)) AS er_fl_chmak_cnahge_email,
                         sum(multiIf((eventlabel = 'NewRegSmsWrongCode') AND match(eventAction, '(FL|P_AGENT)') AND
                                     match(pagepath,
                                           '(/login/$|/login$|offer-placement|/login/client$|/login/professional$|/login/partner$|/reset-password)'),
                                     uniqueEvents, 0)) AS er_fl_chmak_sms_wrong_code,
                         sum(multiIf((eventlabel = 'NewAuthEmailInput') AND match(eventAction, '(FL|P_AGENT)') AND
                                     match(pagepath,
                                           '(/login/$|/login$|offer-placement|/login/client$|/login/professional$|/login/partner$)'),
                                     uniqueEvents, 0)) AS email_setup_for_auth_old_users,
                         sum(multiIf((eventlabel = 'NewAuthPassWrong') AND match(eventAction, '(FL|P_AGENT)') AND
                                     match(pagepath,
                                           '(/login/$|/login$|offer-placement|/login/client$|/login/professional$|/login/partner$|/reset-password)'),
                                     uniqueEvents, 0)) AS auth_wrong_pass,
                         sum(multiIf((eventlabel = 'NewAuthEmailWrongCode') AND
                                     match(eventAction, '(FL|P_AGENT|PARTNER)') AND match(pagepath,
                                                                                          '(/login/$|/login$|offer-placement|/login/client$|/login/professional$|/login/partner$|/reset-password)'),
                                     uniqueEvents, 0)) AS auth_email_wrong_code,
                         sum(multiIf((eventlabel = 'NewAuthEmailChange') AND match(eventAction, '(FL|P_AGENT)') AND
                                     match(pagepath,
                                           '(/login/$|/login$|offer-placement|/login/client$|/login/professional$|/login/partner$|/reset-password)'),
                                     uniqueEvents, 0)) AS auth_email_change,
                         sum(multiIf((eventlabel = 'NewAuthForgotPass') AND match(eventAction, '(FL|P_AGENT)') AND
                                     match(pagepath,
                                           '(/login/$|/login$|offer-placement|/login/client$|/login/professional$|/login/partner$|/reset-password)'),
                                     uniqueEvents, 0)) AS auth_forgot_pass,
                         sum(multiIf((eventlabel = 'NewAuthPhoneInputRecovery') AND
                                     match(eventAction, '(FL|P_AGENT)') AND match(pagepath,
                                                                                  '(/login/$|/login$|offer-placement|/login/client$|/login/professional$|/login/partner$|/reset-password)'),
                                     uniqueEvents, 0)) AS auth_chmak_fl_phone_recov_input,
                         sum(multiIf((eventlabel = 'NewAuthSmsInputRecovery') AND match(eventAction, '(FL|P_AGENT)') AND
                                     match(pagepath,
                                           '(/login/$|/login$|offer-placement|/login/client$|/login/professional$|/login/partner$|/reset-password)'),
                                     uniqueEvents, 0)) AS auth_chmak_fl_phone_recov_sms_input,
                         sum(multiIf((eventlabel = 'NewAuthEmailInputRecovery') AND match(eventAction, '(PARTNER)') AND
                                     match(pagepath,
                                           '(/login/$|/login$|offer-placement|/login/client$|/login/professional$|/login/partner$|/reset-password)'),
                                     uniqueEvents, 0)) AS auth_partner_email_recov_input,
                         sum(multiIf((eventlabel = 'NewAuthWrongDbPhone') AND match(eventAction, '(FL|P_AGENT)') AND
                                     match(pagepath,
                                           '(/login/$|/login$|offer-placement|/login/client$|/login/professional$|/login/partner$|/reset-password)'),
                                     uniqueEvents, 0)) AS auth_chmak_fl_phone_db_wrong,
                         sum(multiIf((eventlabel = 'NewAuthWrongDbEmail') AND match(eventAction, '(PARTNER)') AND
                                     match(pagepath,
                                           '(/login/$|/login$|offer-placement|/login/client$|/login/professional$|/login/partner$|/reset-password)'),
                                     uniqueEvents, 0)) AS auth_partner_email_db_wrong,
                         sum(multiIf((eventlabel = 'NewAuthPassNewSetup') AND
                                     match(eventAction, '(FL|P_AGENT|PARTNER)') AND match(pagepath,
                                                                                          '(/login/$|/login$|offer-placement|/login/client$|/login/professional$|/login/partner$|/reset-password)'),
                                     uniqueEvents, 0)) AS auth_new_pass_setup
                  FROM ga.RAW_EVENTS
                  WHERE match(eventlabel,
                              '(NewAuthCodeInputRecivery|NewAuthEmailChange|NewAuthEmailInput|NewAuthEmailInputRecovery|NewAuthEmailWrongCode|NewAuthForgotPass|NewAuthPassNewSetup|NewAuthPassWrong|NewAuthPhoneChecked|NewAuthPhoneInputRecovery|NewAuthSmsInputRecovery|NewAuthSuccessful|NewAuthWrongDbEmail|NewAuthWrongDbPhone|NewRegEmailChange|NewRegEmailWrongCode|NewRegPhoneChange|NewRegSmsWrongCode|NewRegWrongSmsPhoneChange|NewReg_AuthRoleChoose|NewRegPhoneChecked|NewRegSmsInput|NewRegPassSetup|NewRegSuccess|NewRegEmailInput|NewRegSuccessful|NewRegRequestSend|NewRegRequestStart|NewReg_AuthForgotPass)')
                    AND ((date >= (SELECT max(date) + 1
                                   FROM messnerav.REG_AUTH_TECH_REPORT)) AND (date <= yesterday()))
                    AND match(eventAction, '(FL|P_AGENT|PARTNER)')
                    AND match(pagepath,
                              '(/login/$|/login$|offer-placement|/login/client$|/login/professional$|/login/partner$|/reset-password)')
                  GROUP BY date,
                           eventAction
                  ORDER BY date ASC,
                           eventAction ASC
                  )
         ) AS t2 ON t1.Q1 = t2.Q2 ''', with_column_types = True)

In [ ]:
col = []
for i in columns:
    col.append(i[0])
res = pd.DataFrame(re,columns=col)
res.head(5)

In [ ]:
client.insert_dataframe("INSERT INTO messnerav.REG_AUTH_TECH_REPORT VALUES", res)